# 610 WorkFlow Gerencial - Nicolas Horn (v8)

**Hiperparametros optimizados (6 params con BO 250 iters):**
- num_leaves = 289
- min_data_in_leaf = 279
- max_depth = 10
- lambda_l1 = 0.529 (regularizacion L1)
- lambda_l2 = 3.062 (regularizacion L2)
- min_gain_to_split = 0.070
- num_iterations = 463

**Parametros fijos mejorados:**
- feature_fraction = 0.8 (vs 0.5)
- bagging_fraction = 0.8
- bagging_freq = 1

**Exclusion de variables leaky:**
- numero_de_cliente (excluido)
- foto_mes (excluido)

**Feature Engineering:**
- Catastrophe Analysis (13 vars -> NA)
- Data Drifting por IPC
- lags (1,2,3) + deltas (1,2,3) + trends (3,6)

**Validacion:** 202107 (ganancia local)
**Cortes Kaggle:** 850, 950, 1050, 1150
**5 semillas:** 153929, 838969, 922081, 795581, 194609

## Seteo Google Colab (Python3)

In [1]:
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -sf "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json ~/.kaggle 2>/dev/null || true
chmod 600 ~/.kaggle/kaggle.json 2>/dev/null || true

mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets

webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"
archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget $webfiles/$archivo -O $destino_bucket/$archivo
fi

if ! test -f $destino_local/$archivo; then
  cp $destino_bucket/$archivo $destino_local/$archivo
fi

ls -lh $destino_local/$archivo

-rw------- 1 root root 17M Nov 30 23:00 /content/datasets/gerencial_competencia_2025.csv.gz


## Inicializacion R

**Cambiar Runtime a R**

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

In [1]:
rm(list=ls(all.names=TRUE))
gc(full=TRUE, verbose=FALSE)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,657208,35.1,1454491,77.7,1323974,70.8
Vcells,1220119,9.4,8388608,64.0,1975136,15.1


In [2]:
require("data.table")
if(!require("R.utils")) install.packages("R.utils")
require("R.utils")
if(!require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

Loading required package: data.table

Loading required package: R.utils

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.utils’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.




## Parametros Globales

In [3]:
PARAM_GLOBAL <- list()
PARAM_GLOBAL$experimento_base <- 6180
PARAM_GLOBAL$dataset <- "gerencial_competencia_2025.csv.gz"

PARAM_GLOBAL$semillas <- c(153929, 838969, 922081, 795581, 194609)

# Cortes para Kaggle (espaciados)
PARAM_GLOBAL$kaggle_cortes <- c(850, 950, 1050, 1150)

# Variables a excluir (evitan data leakage)
PARAM_GLOBAL$excluir_campos <- c("numero_de_cliente", "foto_mes")

resultados_totales <- list()

cat("Configuracion v8:\n")
cat("  Excluir:", paste(PARAM_GLOBAL$excluir_campos, collapse=", "), "\n")
cat("  Cortes:", paste(PARAM_GLOBAL$kaggle_cortes, collapse=", "), "\n")
cat("  Semillas:", length(PARAM_GLOBAL$semillas), "\n")

Configuracion v8:
  Excluir: numero_de_cliente, foto_mes 
  Cortes: 850, 950, 1050, 1150 
  Semillas: 5 


## Indices IPC

In [4]:
vfoto_mes <- c(
  202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106, 202107
)

vIPC <- c(
  1.2118694724, 1.1881073259,
  1.1693969743, 1.1375456949, 1.1065619600,
  1.0681100000, 1.0370000000, 1.0000000000,
  0.9680542110, 0.9344152616, 0.8882274350,
  0.8532444140, 0.8251880213, 0.8003763543,
  0.7763107219
)

tb_indices <- data.table(foto_mes = vfoto_mes, IPC = vIPC)
print(tb_indices)

    foto_mes       IPC
       <num>     <num>
 1:   202005 1.2118695
 2:   202006 1.1881073
 3:   202007 1.1693970
 4:   202008 1.1375457
 5:   202009 1.1065620
 6:   202010 1.0681100
 7:   202011 1.0370000
 8:   202012 1.0000000
 9:   202101 0.9680542
10:   202102 0.9344153
11:   202103 0.8882274
12:   202104 0.8532444
13:   202105 0.8251880
14:   202106 0.8003764
15:   202107 0.7763107


## Funcion Tendencia

In [5]:
calc_slope_fast <- function(y) {
  n <- length(y)
  valid <- !is.na(y)
  n_valid <- sum(valid)
  if (n_valid < 2) return(NA_real_)

  x <- 1:n
  x_valid <- x[valid]
  y_valid <- y[valid]

  sum_x <- sum(x_valid)
  sum_y <- sum(y_valid)
  sum_xy <- sum(x_valid * y_valid)
  sum_x2 <- sum(x_valid^2)

  denom <- n_valid * sum_x2 - sum_x^2
  if (denom == 0) return(NA_real_)

  (n_valid * sum_xy - sum_x * sum_y) / denom
}

## Hiperparametros Optimizados (BO 250 iters)

Estos hiperparametros fueron encontrados localmente con Bayesian Optimization extensa (250 iteraciones).

In [6]:
# Hiperparametros optimizados con BO (250 iteraciones)

MEJORES_HIPERPARAMETROS <- list(
  # Optimizados por BO (6 parametros)
  num_leaves = 289,
  min_data_in_leaf = 279,
  max_depth = 10,
  lambda_l1 = 0.5291058,
  lambda_l2 = 3.061922,
  min_gain_to_split = 0.07016705,
  num_iterations = 463
)

cat("Hiperparametros optimizados:\n")
for (param in names(MEJORES_HIPERPARAMETROS)) {
  cat("  ", param, "=", MEJORES_HIPERPARAMETROS[[param]], "\n")
}

Hiperparametros optimizados:
   num_leaves = 289 
   min_data_in_leaf = 279 
   max_depth = 10 
   lambda_l1 = 0.5291058 
   lambda_l2 = 3.061922 
   min_gain_to_split = 0.07016705 
   num_iterations = 463 


## Loop Principal - 5 Semillas

In [7]:
for (seed_idx in 1:length(PARAM_GLOBAL$semillas)) {

  cat("\n\n========================================\n")
  cat("SEMILLA", seed_idx, "de", length(PARAM_GLOBAL$semillas), "\n")
  cat("Semilla:", PARAM_GLOBAL$semillas[seed_idx], "\n")
  cat("========================================\n\n")

  inicio_seed <- Sys.time()

  PARAM <- list()
  PARAM$semilla_primigenia <- PARAM_GLOBAL$semillas[seed_idx]
  PARAM$experimento <- PARAM_GLOBAL$experimento_base + seed_idx - 1
  PARAM$dataset <- PARAM_GLOBAL$dataset

  # Carpeta del Experimento
  if (!dir.exists("/content/buckets/b1/exp")) {
    dir.create("/content/buckets/b1/exp", showWarnings = FALSE, recursive = TRUE)
  }

  setwd("/content/buckets/b1/exp")
  experimento_folder <- paste0("WF", PARAM$experimento, "_seed", seed_idx, "_v8")
  dir.create(experimento_folder, showWarnings=FALSE)
  setwd(paste0("/content/buckets/b1/exp/", experimento_folder))
  dir.create("kaggle", showWarnings=FALSE)

  cat("Carpeta:", experimento_folder, "\n\n")

  # Carga del dataset
  cat("Cargando dataset...\n")
  dataset <- fread(paste0("/content/datasets/", PARAM$dataset))
  cat("Dataset:", nrow(dataset), "x", ncol(dataset), "\n\n")

  # Catastrophe Analysis
  cat("Catastrophe Analysis (13 variables -> NA)...\n")
  dataset[foto_mes==202006, internet:=NA]
  dataset[foto_mes==202006, mrentabilidad:=NA]
  dataset[foto_mes==202006, mrentabilidad_annual:=NA]
  dataset[foto_mes==202006, mcomisiones:=NA]
  dataset[foto_mes==202006, mactivos_margen:=NA]
  dataset[foto_mes==202006, mpasivos_margen:=NA]
  dataset[foto_mes==202006, mcuentas_saldo:=NA]
  dataset[foto_mes==202006, ctarjeta_visa_transacciones:=NA]
  dataset[foto_mes==202006, mtarjeta_visa_consumo:=NA]
  dataset[foto_mes==202006, mtarjeta_master_consumo:=NA]
  dataset[foto_mes==202006, ccallcenter_transacciones:=NA]
  dataset[foto_mes==202006, chomebanking_transacciones:=NA]
  dataset[foto_mes==202006, ctarjeta_master_transacciones:=NA]

  # Data Drifting - IPC
  cat("Data Drifting (IPC)...\n")
  campos_monetarios <- colnames(dataset)[colnames(dataset) %like% "^m"]
  dataset[tb_indices, on = "foto_mes", (campos_monetarios) := .SD * i.IPC, .SDcols = campos_monetarios]

  # FE Intra-mes
  cat("FE Intra-mes...\n")
  dataset[, kmes := foto_mes %% 100]
  if("mpayroll" %in% colnames(dataset) & "cliente_edad" %in% colnames(dataset))
    dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

  # FE Historico (extendido: lags 1,2,3 + deltas 1,2,3 + trends 3,6)
  cat("FE Historico (lags 1,2,3 + deltas 1,2,3 + trends 3,6)...\n")
  inicio_fe <- Sys.time()
  setorder(dataset, numero_de_cliente, foto_mes)

  cols_lagueables <- setdiff(colnames(dataset), c("numero_de_cliente", "foto_mes", "clase_ternaria"))

  # Lags 1, 2, 3
  cat("  Lags...\n")
  dataset[, paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"), by = numero_de_cliente, .SDcols = cols_lagueables]
  dataset[, paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"), by = numero_de_cliente, .SDcols = cols_lagueables]
  dataset[, paste0(cols_lagueables, "_lag3") := shift(.SD, 3, NA, "lag"), by = numero_de_cliente, .SDcols = cols_lagueables]

  # Deltas 1, 2, 3
  cat("  Deltas...\n")
  for (vcol in cols_lagueables) {
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
    dataset[, paste0(vcol, "_delta3") := get(vcol) - get(paste0(vcol, "_lag3"))]
  }

  # Trends 3 y 6
  cat("  Trends...\n")
  for (col in cols_lagueables) {
    dataset[, paste0(col, "_trend_3") := frollapply(get(col), 3, calc_slope_fast, align="right"), by = numero_de_cliente]
  }

  for (col in cols_lagueables) {
    dataset[, paste0(col, "_trend_6") := frollapply(get(col), 6, calc_slope_fast, align="right"), by = numero_de_cliente]
  }

  cat("FE completado en", round(difftime(Sys.time(), inicio_fe, units="mins"), 1), "min\n")
  cat("Dataset:", ncol(dataset), "columnas\n\n")

  # Training Strategy - Validar en 202107
  cat("Training Strategy (validar 202107)...\n")
  PARAM$trainingstrategy <- list()
  PARAM$trainingstrategy$final_train <- c(202106, 202105, 202104, 202103, 202102, 202101, 202012, 202011, 202010, 202009, 202008, 202007, 202006, 202005)
  PARAM$trainingstrategy$future <- c(202107)

  dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1, 0)]

  # EXCLUSION DE VARIABLES LEAKY
  campos_buenos <- setdiff(colnames(dataset), c("clase_ternaria", "clase01", "azar"))
  campos_buenos <- setdiff(campos_buenos, PARAM_GLOBAL$excluir_campos)

  cat("Variables excluidas:", paste(PARAM_GLOBAL$excluir_campos, collapse=", "), "\n")

  set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
  dataset[, azar := runif(nrow(dataset))]
  dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train]

  cat("Features para entrenar:", length(campos_buenos), "\n\n")

  # Parametros combinados: fijos mejorados + optimizados
  cat("Configurando modelo con hiperparametros optimizados...\n")

  param_final <- list(
    # Basicos
    objective = "binary",
    metric = "auc",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    verbosity = -100,
    force_row_wise = TRUE,
    seed = PARAM$semilla_primigenia,

    # Fijos mejorados
    max_bin = 31,
    learning_rate = 0.03,
    feature_fraction = 0.8,
    bagging_fraction = 0.8,
    bagging_freq = 1,

    # Optimizados por BO (250 iters)
    num_leaves = MEJORES_HIPERPARAMETROS$num_leaves,
    min_data_in_leaf = MEJORES_HIPERPARAMETROS$min_data_in_leaf,
    max_depth = MEJORES_HIPERPARAMETROS$max_depth,
    lambda_l1 = MEJORES_HIPERPARAMETROS$lambda_l1,
    lambda_l2 = MEJORES_HIPERPARAMETROS$lambda_l2,
    min_gain_to_split = MEJORES_HIPERPARAMETROS$min_gain_to_split,
    num_iterations = MEJORES_HIPERPARAMETROS$num_iterations
  )

  cat("Parametros clave:\n")
  cat("  num_leaves =", param_final$num_leaves, "\n")
  cat("  min_data_in_leaf =", param_final$min_data_in_leaf, "\n")
  cat("  max_depth =", param_final$max_depth, "\n")
  cat("  lambda_l1 =", param_final$lambda_l1, "\n")
  cat("  lambda_l2 =", param_final$lambda_l2, "\n")
  cat("  feature_fraction =", param_final$feature_fraction, "\n")
  cat("  bagging_fraction =", param_final$bagging_fraction, "\n")
  cat("  num_iterations =", param_final$num_iterations, "\n\n")

  # Final Training
  cat("Entrenando modelo...\n")

  dfinal_train <- lgb.Dataset(
    data = data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with = FALSE]),
    label = dataset[fold_final_train == TRUE, clase01],
    free_raw_data = TRUE
  )

  inicio_train <- Sys.time()
  final_model <- lgb.train(data = dfinal_train, param = param_final, verbose = -100)
  cat("Modelo entrenado en", round(difftime(Sys.time(), inicio_train, units="mins"), 1), "min\n\n")

  lgb.save(final_model, "modelo.txt")
  fwrite(as.data.table(lgb.importance(final_model)), file = "impo.txt", sep = "\t")

  # Scoring 202107
  cat("Scoring 202107...\n")
  dfuture <- dataset[foto_mes %in% PARAM$trainingstrategy$future]
  prediccion <- predict(final_model, data.matrix(dfuture[, campos_buenos, with = FALSE]))

  tb_prediccion <- dfuture[, list(numero_de_cliente)]
  tb_prediccion[, prob := prediccion]
  fwrite(tb_prediccion, file = "prediccion.txt", sep = "\t")

  # Curva de Ganancia (validacion local)
  tb_prediccion[, clase_ternaria := dfuture$clase_ternaria]
  tb_prediccion[, ganancia := -3000.0]
  tb_prediccion[clase_ternaria == "BAJA+2", ganancia := 117000.0]

  setorder(tb_prediccion, -prob)
  tb_prediccion[, gan_acum := cumsum(ganancia)]
  tb_prediccion[, gan_suavizada := frollmean(gan_acum, 400, align="center", na.rm=TRUE)]

  resultado <- list()
  resultado$ganancia_suavizada_max <- max(tb_prediccion$gan_suavizada, na.rm=TRUE)
  resultado$envios <- which.max(tb_prediccion$gan_suavizada)
  resultado$semilla <- PARAM$semilla_primigenia
  resultado$seed_idx <- seed_idx

  # Ganancia en los cortes especificos
  resultado$ganancia_cortes <- list()
  for (corte in PARAM_GLOBAL$kaggle_cortes) {
    resultado$ganancia_cortes[[as.character(corte)]] <- tb_prediccion[corte, gan_acum]
  }

  fwrite(tb_prediccion, file = "ganancias.txt", sep = "\t")

  # Grafico
  tb_prediccion[, envios_num := .I]
  pdf("curva_de_ganancia.pdf")
  plot(x = tb_prediccion$envios_num, y = tb_prediccion$gan_acum, type = "l", col = "gray",
       xlim = c(0, 6000), ylim = c(0, 8000000),
       main = paste0("Seed ", seed_idx, " (v8) - Gan=", as.integer(resultado$ganancia_suavizada_max)),
       xlab = "Envios", ylab = "Ganancia", panel.first = grid())
  abline(v = PARAM_GLOBAL$kaggle_cortes, col = "red", lty = 2)
  dev.off()

  # Generar CSVs para Kaggle (sin enviar)
  cat("Generando CSVs para Kaggle (cortes:", paste(PARAM_GLOBAL$kaggle_cortes, collapse=", "), ")...\n")
  for (envios in PARAM_GLOBAL$kaggle_cortes) {
    tb_prediccion[, Predicted := 0L]
    tb_prediccion[1:envios, Predicted := 1L]
    archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")
    fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)], file = archivo_kaggle, sep = ",")
    cat("  ->", archivo_kaggle, "| Ganancia local:", as.integer(tb_prediccion[envios, gan_acum]), "\n")
  }

  # Guardar
  if(!require("yaml")) install.packages("yaml")
  require("yaml")
  PARAM$resultado <- resultado
  PARAM$mejores_hiperparametros <- MEJORES_HIPERPARAMETROS
  write_yaml(PARAM, file = "PARAM.yml")

  resultados_totales[[seed_idx]] <- resultado

  fin_seed <- Sys.time()
  duracion <- as.numeric(difftime(fin_seed, inicio_seed, units = "mins"))

  rm(dataset, dfinal_train, final_model, tb_prediccion, dfuture)
  gc(full = TRUE, verbose = FALSE)

  cat("\n========================================\n")
  cat("Semilla", seed_idx, "completada en", round(duracion, 1), "min\n")
  cat("Ganancia max:", formatC(resultado$ganancia_suavizada_max, format="f", big.mark=",", digits=0), "\n")
  cat("Envios optimos:", resultado$envios, "\n")
  cat("========================================\n")
}

cat("\n*** TODAS LAS SEMILLAS PROCESADAS ***\n")



SEMILLA 1 de 5 
Semilla: 153929 

Carpeta: WF6180_seed1_v8 

Cargando dataset...
Dataset: 273666 x 32 

Catastrophe Analysis (13 variables -> NA)...
Data Drifting (IPC)...
FE Intra-mes...
FE Historico (lags 1,2,3 + deltas 1,2,3 + trends 3,6)...
  Lags...
  Deltas...
  Trends...
FE completado en 3.6 min
Dataset: 291 columnas

Training Strategy (validar 202107)...
Variables excluidas: numero_de_cliente, foto_mes 
Features para entrenar: 288 

Configurando modelo con hiperparametros optimizados...
Parametros clave:
  num_leaves = 289 
  min_data_in_leaf = 279 
  max_depth = 10 
  lambda_l1 = 0.5291058 
  lambda_l2 = 3.061922 
  feature_fraction = 0.8 
  bagging_fraction = 0.8 
  num_iterations = 463 

Entrenando modelo...
Modelo entrenado en 2.1 min

Scoring 202107...
Generando CSVs para Kaggle (cortes: 850, 950, 1050, 1150 )...
  -> ./kaggle/KA6180_850.csv | Ganancia local: 5970000 
  -> ./kaggle/KA6180_950.csv | Ganancia local: 6510000 
  -> ./kaggle/KA6180_1050.csv | Ganancia local: 

Loading required package: yaml




Semilla 1 completada en 6 min
Ganancia max: 6,718,800 
Envios optimos: 1519 


SEMILLA 2 de 5 
Semilla: 838969 

Carpeta: WF6181_seed2_v8 

Cargando dataset...
Dataset: 273666 x 32 

Catastrophe Analysis (13 variables -> NA)...
Data Drifting (IPC)...
FE Intra-mes...
FE Historico (lags 1,2,3 + deltas 1,2,3 + trends 3,6)...
  Lags...
  Deltas...
  Trends...
FE completado en 3.7 min
Dataset: 291 columnas

Training Strategy (validar 202107)...
Variables excluidas: numero_de_cliente, foto_mes 
Features para entrenar: 288 

Configurando modelo con hiperparametros optimizados...
Parametros clave:
  num_leaves = 289 
  min_data_in_leaf = 279 
  max_depth = 10 
  lambda_l1 = 0.5291058 
  lambda_l2 = 3.061922 
  feature_fraction = 0.8 
  bagging_fraction = 0.8 
  num_iterations = 463 

Entrenando modelo...
Modelo entrenado en 2.2 min

Scoring 202107...
Generando CSVs para Kaggle (cortes: 850, 950, 1050, 1150 )...
  -> ./kaggle/KA6181_850.csv | Ganancia local: 5730000 
  -> ./kaggle/KA6181_950.c

## Resumen Final

In [8]:
setwd("/content/buckets/b1/exp")

tb_resumen <- data.table(
  seed_idx = sapply(resultados_totales, function(x) x$seed_idx),
  semilla = sapply(resultados_totales, function(x) x$semilla),
  ganancia = sapply(resultados_totales, function(x) x$ganancia_suavizada_max),
  envios = sapply(resultados_totales, function(x) x$envios)
)

# Agregar ganancias por corte
for (corte in PARAM_GLOBAL$kaggle_cortes) {
  col_name <- paste0("gan_", corte)
  tb_resumen[, (col_name) := sapply(resultados_totales, function(x) x$ganancia_cortes[[as.character(corte)]])]
}

tb_resumen[, rank := rank(-ganancia)]

cat("\n========================================\n")
cat("RESUMEN FINAL - v8\n")
cat("========================================\n\n")
print(tb_resumen)

cat("\nESTADISTICAS:\n")
cat("Ganancia promedio:", formatC(mean(tb_resumen$ganancia), format="f", big.mark=",", digits=0), "\n")
cat("Ganancia maxima:", formatC(max(tb_resumen$ganancia), format="f", big.mark=",", digits=0), "\n")
cat("Ganancia minima:", formatC(min(tb_resumen$ganancia), format="f", big.mark=",", digits=0), "\n")

cat("\nGANANCIA PROMEDIO POR CORTE:\n")
for (corte in PARAM_GLOBAL$kaggle_cortes) {
  col_name <- paste0("gan_", corte)
  cat("  Corte", corte, ":", formatC(mean(tb_resumen[[col_name]]), format="f", big.mark=",", digits=0), "\n")
}

cat("\nCOMPARACION:\n")
cat("  WF6300 original (Kaggle): 4,587,000\n")
cat("  v8 promedio (local Jul):", formatC(mean(tb_resumen$ganancia), format="f", big.mark=",", digits=0), "\n")

fwrite(tb_resumen, file = paste0("resumen_v8_exp", PARAM_GLOBAL$experimento_base, ".txt"), sep = "\t")
saveRDS(resultados_totales, file = paste0("resultados_v8_exp", PARAM_GLOBAL$experimento_base, ".rds"))



RESUMEN FINAL - v8

   seed_idx semilla ganancia envios gan_850 gan_950 gan_1050 gan_1150  rank
      <int>   <num>    <num>  <int>   <num>   <num>    <num>    <num> <num>
1:        1  153929  6718800   1519 5970000 6510000  6690000  6510000     3
2:        2  838969  6900000   1497 5730000 6150000  6210000  6510000     2
3:        3  922081  6513300   1698 6090000 5910000  5970000  6150000     5
4:        4  795581  7312800   1559 6090000 6270000  6450000  6510000     1
5:        5  194609  6715500   1515 6450000 6270000  6450000  6630000     4

ESTADISTICAS:
Ganancia promedio: 6,832,080 
Ganancia maxima: 7,312,800 
Ganancia minima: 6,513,300 

GANANCIA PROMEDIO POR CORTE:
  Corte 850 : 6,066,000 
  Corte 950 : 6,222,000 
  Corte 1050 : 6,354,000 
  Corte 1150 : 6,462,000 

COMPARACION:
  WF6300 original (Kaggle): 4,587,000
  v8 promedio (local Jul): 6,832,080 

CSVs para Kaggle generados en cada carpeta WF*/kaggle/
Cuando se reseteen los submissions, subir manualmente.


In [10]:
# Generar CSVs con cortes optimos para las 5 semillas
  # (usando los archivos ganancias.txt ya guardados)

  cat("Generando CSVs con cortes optimos...\n\n")

  for (seed_idx in 1:length(PARAM_GLOBAL$semillas)) {

    experimento <- PARAM_GLOBAL$experimento_base + seed_idx - 1
    carpeta <- paste0("/content/buckets/b1/exp/WF", experimento, "_seed", seed_idx, "_v8")

    # Leer predicciones guardadas
    tb_pred <- fread(paste0(carpeta, "/ganancias.txt"))
    setorder(tb_pred, -prob)

    # Obtener el corte optimo de esta semilla
    corte_optimo <- resultados_totales[[seed_idx]]$envios

    # Generar CSV
    tb_pred[, Predicted := 0L]
    tb_pred[1:corte_optimo, Predicted := 1L]

    archivo <- paste0(carpeta, "/kaggle/KA", experimento, "_", corte_optimo, ".csv")
    fwrite(tb_pred[, list(numero_de_cliente, Predicted)], file = archivo, sep = ",")

    # Mostrar ganancia en ese corte
    ganancia_optimo <- tb_pred[corte_optimo, gan_acum]

    cat(sprintf("Seed %d: Corte %d -> %s | Archivo: KA%d_%d.csv\n",
      seed_idx,
      corte_optimo,
      formatC(ganancia_optimo, format="f", big.mark=",", digits=0),
      experimento,
      corte_optimo
    ))
  }

  cat("\n*** CSVs con cortes optimos generados ***\n")

Generando CSVs con cortes optimos...

Seed 1: Corte 1519 -> 6,723,000 | Archivo: KA6180_1519.csv
Seed 2: Corte 1497 -> 6,909,000 | Archivo: KA6181_1497.csv
Seed 3: Corte 1698 -> 6,426,000 | Archivo: KA6182_1698.csv
Seed 4: Corte 1559 -> 7,323,000 | Archivo: KA6183_1559.csv
Seed 5: Corte 1515 -> 6,855,000 | Archivo: KA6184_1515.csv

*** CSVs con cortes optimos generados ***


In [11]:
# Generar CSVs para Kaggle (202109) usando los modelos guardados
  # Primero hay que recargar el dataset y aplicar el mismo FE

  cat("Generando predicciones para Kaggle (202109)...\n\n")

  # Cargar dataset fresco
  cat("Cargando dataset...\n")
  dataset_kaggle <- fread(paste0("/content/datasets/", PARAM_GLOBAL$dataset))

  # Aplicar Catastrophe Analysis
  cat("Aplicando Catastrophe Analysis...\n")
  dataset_kaggle[foto_mes==202006, internet:=NA]
  dataset_kaggle[foto_mes==202006, mrentabilidad:=NA]
  dataset_kaggle[foto_mes==202006, mrentabilidad_annual:=NA]
  dataset_kaggle[foto_mes==202006, mcomisiones:=NA]
  dataset_kaggle[foto_mes==202006, mactivos_margen:=NA]
  dataset_kaggle[foto_mes==202006, mpasivos_margen:=NA]
  dataset_kaggle[foto_mes==202006, mcuentas_saldo:=NA]
  dataset_kaggle[foto_mes==202006, ctarjeta_visa_transacciones:=NA]
  dataset_kaggle[foto_mes==202006, mtarjeta_visa_consumo:=NA]
  dataset_kaggle[foto_mes==202006, mtarjeta_master_consumo:=NA]
  dataset_kaggle[foto_mes==202006, ccallcenter_transacciones:=NA]
  dataset_kaggle[foto_mes==202006, chomebanking_transacciones:=NA]
  dataset_kaggle[foto_mes==202006, ctarjeta_master_transacciones:=NA]

  # Aplicar Data Drifting IPC
  cat("Aplicando Data Drifting (IPC)...\n")
  campos_monetarios <- colnames(dataset_kaggle)[colnames(dataset_kaggle) %like% "^m"]
  dataset_kaggle[tb_indices, on = "foto_mes", (campos_monetarios) := .SD * i.IPC, .SDcols = campos_monetarios]

  # FE Intra-mes
  cat("Aplicando FE Intra-mes...\n")
  dataset_kaggle[, kmes := foto_mes %% 100]
  if("mpayroll" %in% colnames(dataset_kaggle) & "cliente_edad" %in% colnames(dataset_kaggle))
    dataset_kaggle[, mpayroll_sobre_edad := mpayroll / cliente_edad]

  # FE Historico
  cat("Aplicando FE Historico (lags, deltas, trends)...\n")
  setorder(dataset_kaggle, numero_de_cliente, foto_mes)
  cols_lagueables <- setdiff(colnames(dataset_kaggle), c("numero_de_cliente", "foto_mes", "clase_ternaria"))

  # Lags
  dataset_kaggle[, paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"), by = numero_de_cliente, .SDcols = cols_lagueables]
  dataset_kaggle[, paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"), by = numero_de_cliente, .SDcols = cols_lagueables]
  dataset_kaggle[, paste0(cols_lagueables, "_lag3") := shift(.SD, 3, NA, "lag"), by = numero_de_cliente, .SDcols = cols_lagueables]

  # Deltas
  for (vcol in cols_lagueables) {
    dataset_kaggle[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset_kaggle[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
    dataset_kaggle[, paste0(vcol, "_delta3") := get(vcol) - get(paste0(vcol, "_lag3"))]
  }

  # Trends
  for (col in cols_lagueables) {
    dataset_kaggle[, paste0(col, "_trend_3") := frollapply(get(col), 3, calc_slope_fast, align="right"), by = numero_de_cliente]
  }
  for (col in cols_lagueables) {
    dataset_kaggle[, paste0(col, "_trend_6") := frollapply(get(col), 6, calc_slope_fast, align="right"), by = numero_de_cliente]
  }

  cat("FE completado. Columnas:", ncol(dataset_kaggle), "\n\n")

  # Filtrar solo 202109
  df_202109 <- dataset_kaggle[foto_mes == 202109]
  cat("Registros en 202109:", nrow(df_202109), "\n\n")

  # Campos para prediccion (excluir leaky)
  campos_buenos <- setdiff(colnames(dataset_kaggle), c("clase_ternaria", "clase01", "azar"))
  campos_buenos <- setdiff(campos_buenos, PARAM_GLOBAL$excluir_campos)

  # Generar predicciones para cada semilla
  cat("Generando CSVs para Kaggle...\n\n")

  for (seed_idx in 1:length(PARAM_GLOBAL$semillas)) {

    experimento <- PARAM_GLOBAL$experimento_base + seed_idx - 1
    carpeta <- paste0("/content/buckets/b1/exp/WF", experimento, "_seed", seed_idx, "_v8")

    # Cargar modelo guardado
    modelo <- lgb.load(paste0(carpeta, "/modelo.txt"))

    # Predecir sobre 202109
    prediccion <- predict(modelo, data.matrix(df_202109[, campos_buenos, with = FALSE]))

    # Crear tabla de prediccion
    tb_kaggle <- df_202109[, list(numero_de_cliente)]
    tb_kaggle[, prob := prediccion]
    setorder(tb_kaggle, -prob)

    # Obtener corte optimo de Julio (como referencia)
    corte_optimo <- resultados_totales[[seed_idx]]$envios

    # Generar CSVs con los cortes definidos + el optimo
    cortes_a_generar <- unique(c(PARAM_GLOBAL$kaggle_cortes, corte_optimo))

    for (corte in cortes_a_generar) {
      tb_kaggle[, Predicted := 0L]
      tb_kaggle[1:corte, Predicted := 1L]

      archivo <- paste0(carpeta, "/kaggle/KA", experimento, "_", corte, "_kaggle.csv")
      fwrite(tb_kaggle[, list(numero_de_cliente, Predicted)], file = archivo, sep = ",")
    }

    cat(sprintf("Seed %d: Generados %d CSVs (cortes: %s)\n",
      seed_idx,
      length(cortes_a_generar),
      paste(sort(cortes_a_generar), collapse=", ")
    ))
  }

Generando predicciones para Kaggle (202109)...

Cargando dataset...
Aplicando Catastrophe Analysis...
Aplicando Data Drifting (IPC)...
Aplicando FE Intra-mes...
Aplicando FE Historico (lags, deltas, trends)...
FE completado. Columnas: 291 

Registros en 202109: 16551 

Generando CSVs para Kaggle...

Seed 1: Generados 5 CSVs (cortes: 850, 950, 1050, 1150, 1519)
Seed 2: Generados 5 CSVs (cortes: 850, 950, 1050, 1150, 1497)
Seed 3: Generados 5 CSVs (cortes: 850, 950, 1050, 1150, 1698)
Seed 4: Generados 5 CSVs (cortes: 850, 950, 1050, 1150, 1559)
Seed 5: Generados 5 CSVs (cortes: 850, 950, 1050, 1150, 1515)


In [9]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Nov 30 11:39:46 PM 2025"